In [1]:
from __future__ import print_function

from rdkit import Chem
from rdkit.Chem import AllChem

In [20]:
## Aldehyde & Ketone

METAL = AllChem.ReactionFromSmarts('[C:1]=[O:2].[C,c:3][Mg+,Li:4]>>[*:3][C:1][O:2]')

RED = AllChem.ReactionFromSmarts('[C:1]=[O:2]>>[C:1][O:2]')
RRED = AllChem.ReactionFromSmarts('[C:1]=[O:2]>>[C:1]') # Clemmensen and Wolff-Kishner

WITTIG = AllChem.ReactionFromSmarts('[C:1]=[O:2].[C:3][P+:4]>>[C:1]=[C:3]')
_YLIDE = AllChem.ReactionFromSmarts('[C:1]=[O:2]>>[C:1][P+](c1ccccc1)(c1ccccc1)(c1ccccc1)')

HYD = AllChem.ReactionFromSmarts('[C:1]=[O:2].[O:3]>>[C:1]([O:2])[O:3]')
CYAN = AllChem.ReactionFromSmarts('[C:1]=[O:2].[C-:3]#[N:4]>>[C:1]([O:2])[C-0:3]#[N:4]')
IMINE = AllChem.ReactionFromSmarts('[C:1]=[O:2].[NH2:3]>>[C:1]=[N:3]')
OXIME = AllChem.ReactionFromSmarts('[C:1]=[O:2].[NH2:3][O:4]>>[C:1]=[N:3][O:4]')
HYDRAZONE = AllChem.ReactionFromSmarts('[C:1]=[O:2].[NH2:3][N:4]>>[C:1]=[N:3][N:4]')

ACETAL = AllChem.ReactionFromSmarts('[C:1]=[O:2].[O:3]>>[C:1]([O:3])[O:3]')
ACETAL_ALT = AllChem.ReactionFromSmarts('[C:1]=[O:2].([O:3].[O:4])>>[C:1]([O:3])[O:4]')

OXI = AllChem.ReactionFromSmarts('[C:1](=[O:2])[H]>>[C:1](=[O:2])O')

carbonyl_reactions_list = [
    
    ('METAL', METAL, ['[Li]C', '[Li]CC', '[Li]CCC', '[Li]CCCC', '[Li]c1ccccc1', \
                '[Br-].[Mg+]C', '[Br-].[Mg+]CC', '[Br-].[Mg+]CCC', '[Br-].[Mg+]CCCC', '[Br-].[Mg+]c1ccccc1'], ['']),
    
    ('RED', RED, [''], ['[H-].[H-].[H-].[H-].[Al+3].[Li+]', '[Na+].[BH4-]', '[H].[H]']),
    ('RRED', RRED, [''], ['[Hg].[Zn].Cl', 'NN.[K+].[OH-]']),
    
    ('WITTIG', WITTIG, [''], ['']),
    
    ('HYD', HYD, ['O'], ['']),
    ('CYAN', CYAN, ['[C-]#N'], ['C#N']),
    ('IMINE', IMINE, [''], ['Cl']),
    ('OXIME', OXIME, ['NO'], ['Cl']),
    ('HYDRAZONE', HYDRAZONE, ['NN', 'NNc1ccccc1', 'NNC(N)=O'], ['Cl']),
    
    ('ACETAL', ACETAL, ['CO.CO', 'CCO.CCO', 'CCCO.CCCO', 'CCCCO.CCCCO'], ['Cl']),
    ('ACETAL_ALT', ACETAL_ALT, ['OCCO'], ['Cl']),
    
    ('OXI', OXI, [''], ['[O-][Cr](=O)(=O)O[Cr](=O)(=O)[O-].[Na+].[Na+].OS(O)(=O)=O', \
                             '[O-][Cr](=O)(=O)O[Cr](=O)(=O)[O-].[K+].[K+].OS(O)(=O)=O', \
                             '[K+].[O-][Mn](=O)(=O)=O']),
    
]

In [23]:
import random, copy

def cano(smiles): # canonicalize smiles by MolToSmiles function
    return Chem.MolToSmiles(Chem.MolFromSmiles(smiles))

def block(ch, smiles):
    return (ch + cano(smiles)) if (smiles != '') else cano(smiles)

# choose carbonyl compound (tmp)
carbonyl_smi = 'CCC(C)CCC(C)=O'
carbonyl = Chem.MolFromSmiles(carbonyl_smi)

# choose another carbonyl compound for Wittig (tmp)
ylide_smi = 'CC(C)=O'
ylide_mol = Chem.MolFromSmiles(ylide_smi)
ylide_product = _YLIDE.RunReactants((ylide_mol,))
ylide = Chem.MolToSmiles(ylide_product[0][0])

# choose amine for imine formation (tmp)
amine_smi = 'NCCCC'

# general reactions
for reaction in carbonyl_reactions_list:
    print(reaction[0])
    for reagent in reaction[2]:
        if reaction[0] == 'IMINE':
            reagent = amine_smi
        if reaction[0] == 'WITTIG':
            reagent = ylide
        if reagent == '': products = reaction[1].RunReactants((carbonyl,))
        else: products = reaction[1].RunReactants((carbonyl, Chem.MolFromSmiles(reagent)))
        if len(products) == 0: continue
        product_smi = [Chem.MolToSmiles(product) for product in products[0]]
        if reaction[0] == 'WITTIG':
            for halogen in ['[Cl-]', '[Br-]', '[I-]']:
                reagent_m = reagent + '.' + halogen
                print(cano(carbonyl_smi) + block('.', reagent_m) + '>' + cano(sub_reagent) + '>' + '.'.join(product_smi))
        else:
            for sub_reagent in reaction[3]:
                print(cano(carbonyl_smi) + block('.', reagent) + '>' + cano(sub_reagent) + '>' + '.'.join(product_smi))

METAL
CCC(C)CCC(C)=O.[Li]C>>CCC(C)CCC(C)(C)O
CCC(C)CCC(C)=O.[Li]CC>>CCC(C)CCC(C)(O)CC
CCC(C)CCC(C)=O.[Li]CCC>>CCCC(C)(O)CCC(C)CC
CCC(C)CCC(C)=O.[Li]CCCC>>CCCCC(C)(O)CCC(C)CC
CCC(C)CCC(C)=O.[Li]c1ccccc1>>CCC(C)CCC(C)(O)c1ccccc1
CCC(C)CCC(C)=O.C[Mg+].[Br-]>>CCC(C)CCC(C)(C)O
CCC(C)CCC(C)=O.CC[Mg+].[Br-]>>CCC(C)CCC(C)(O)CC
CCC(C)CCC(C)=O.CCC[Mg+].[Br-]>>CCCC(C)(O)CCC(C)CC
CCC(C)CCC(C)=O.CCCC[Mg+].[Br-]>>CCCCC(C)(O)CCC(C)CC
CCC(C)CCC(C)=O.[Br-].[Mg+]c1ccccc1>>CCC(C)CCC(C)(O)c1ccccc1
RED
CCC(C)CCC(C)=O>[Al+3].[H-].[H-].[H-].[H-].[Li+]>CCC(C)CCC(C)O
CCC(C)CCC(C)=O>[BH4-].[Na+]>CCC(C)CCC(C)O
CCC(C)CCC(C)=O>[H].[H]>CCC(C)CCC(C)O
RRED
CCC(C)CCC(C)=O>Cl.[Hg].[Zn]>CCCCC(C)CC
CCC(C)CCC(C)=O>NN.[K+].[OH-]>CCCCC(C)CC
WITTIG
CCC(C)CCC(C)=O.CC(C)[P+](c1ccccc1)(c1ccccc1)c1ccccc1.[Cl-]>NN.[K+].[OH-]>CCC(C)CCC(C)=C(C)C
CCC(C)CCC(C)=O.CC(C)[P+](c1ccccc1)(c1ccccc1)c1ccccc1.[Br-]>NN.[K+].[OH-]>CCC(C)CCC(C)=C(C)C
CCC(C)CCC(C)=O.CC(C)[P+](c1ccccc1)(c1ccccc1)c1ccccc1.[I-]>NN.[K+].[OH-]>CCC(C)CCC(C)=C(C)C
HYD
CC